In [1]:
import torch_geometric.nn as gnn
import torch_geometric
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.linalg
from torch_geometric.nn import GATConv
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [3]:
# from torch_geometric.datasets import Planetoid
# name_data = 'Cora'
# dataset = Planetoid(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import WebKB
# name_data = 'Wisconsin'
# dataset = WebKB(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import Planetoid
# name_data = 'CiteSeer'
# dataset = Planetoid(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import WebKB
# name_data = 'Cornell'
# dataset = WebKB(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import WebKB
# name_data = 'Texas'
# dataset = WebKB(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import WikipediaNetwork
# name_data = 'Chameleon'
# dataset = WikipediaNetwork(root= '/tmp/' + name_data, name = name_data)

from torch_geometric.datasets import WikipediaNetwork
name_data = 'Squirrel'
dataset = WikipediaNetwork(root= '/tmp/' + name_data, name = name_data)

In [4]:
data = dataset[0]

edges = data.edge_index

#dimension: num_nodes * num_features
features = data.x

num_nodes = features.shape[0]
num_features = features.shape[1]
#dimension: num_nodes * num_nodes
adj = torch.zeros((num_nodes,num_nodes))
degree = torch.zeros((num_nodes,num_nodes))
for i in range(edges.shape[1]):
    first = edges[0][i]
    second = edges[1][i]
    adj[first][second] = 1
    adj[second][first] = 1
for i in range(num_nodes):
    degree[i][i] = sum(adj[i][:])

In [5]:
def data_sample(num_nodes):
  sample_num = int(num_nodes*0.6)
  sample_lst = np.random.choice(num_nodes,sample_num, replace = False)
  train_mask = [False] * num_nodes
  test_mask = [True] * num_nodes
  for i in sample_lst:
    train_mask[i] = True
    test_mask[i] = False
  train_mask = torch.tensor(train_mask)
  test_mask = torch.tensor(test_mask)
  return train_mask, test_mask


In [6]:
def reset_parameter(model):
    for layer in model.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()


In [7]:
degree = degree.to(device)
adj = adj.to(device)
data = data.to(device)

In [8]:
class FBLayer(nn.Module):
    def __init__(self, dim_in, dim_out):
      super().__init__()
      self.low = nn.Linear(dim_in, dim_out, bias = False)
      gain = nn.init.calculate_gain("relu")
      nn.init.xavier_normal_(self.low.weight, gain)
      self.activation = nn.ReLU()
        
    def forward(self, input, adj, degree):
      lap = degree - adj
      inter = scipy.linalg.fractional_matrix_power(degree.cpu(), (-1/2))
      d_inv = torch.from_numpy(inter).to(device)
      Llp = torch.mm(torch.mm(d_inv, lap), d_inv)
      Hl = torch.mm(Llp, self.activation(self.low(input)))
      return Hl

    def reset_parameter():
      gain = nn.init.calculate_gain("relu")
      nn.init.xavier_normal_(self.low.weight, gain)


In [9]:
class GAT(torch.nn.Module):
    def __init__(self, num_head, nhid):
        super().__init__()
        self.gc1 = GATConv(dataset.num_node_features, nhid, heads=num_head)
        self.gc2 = GATConv(nhid*num_head, dataset.num_classes)
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.gc1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, training=self.training)
        x = self.gc2(x, edge_index)
        return F.log_softmax(x, dim=1)
    

In [10]:
class FBGAT(torch.nn.Module):
    def __init__(self, num_head, nhid):
      super().__init__()
      self.fb1 = FBLayer(dataset.num_node_features, nhid*num_head)
      self.fb2 = FBLayer(nhid*num_head ,dataset.num_classes)
      self.conv1 = GATConv(dataset.num_node_features, nhid, heads = num_head)
      self.conv2 = GATConv(nhid*num_head, dataset.num_classes)
      self.aL_1 = torch.nn.Parameter(torch.rand(1))
      self.aH_1 = torch.nn.Parameter(torch.rand(1))
      self.aL_2 = torch.nn.Parameter(torch.rand(1))
      self.aH_2 = torch.nn.Parameter(torch.rand(1))

    def forward(self, data, adj, degree):
      input, edge_index = data.x, data.edge_index
      x_1 = self.fb1(input, adj, degree) #high pass
      y_1 = self.conv1(input, edge_index) #low pass layer
      z_1 = self.aL_1 * x_1 + self.aH_1 * y_1
      z_1 = F.elu(z_1)
      z_1 = F.dropout(z_1, training=self.training)
      x_2 = self.fb2(z_1, adj, degree)
      y_2 = self.conv2(z_1, edge_index)
      z_2 = self.aL_2 * x_2 + self.aH_2 * y_2

      return F.log_softmax(z_2, dim=1)

In [11]:
acc_list_2 = []
for i in range(5):
    model_2 = GAT(48, 8).to(device)
    optimizer_2 = torch.optim.Adam(model_2.parameters(), lr=0.05, weight_decay=5e-5)
    model_2.train()
    print("Number of Trial: " + str(i))
    reset_parameter(model_2)
    train_mask, test_mask = data_sample(num_nodes)
    for epoch in range(100):
        optimizer_2.zero_grad()
        out = model_2(data)
        loss = F.nll_loss(out[train_mask], data.y[train_mask]) 
        loss.backward()
        optimizer_2.step()
    model_2.eval()
    _, pred = model_2(data).max(dim=1)
    correct = int(pred[test_mask].eq(data.y[test_mask]).sum().item())
    acc = correct / int(test_mask.sum())
    acc_list_2.append(acc)

Number of Trial: 0


RuntimeError: CUDA out of memory. Tried to allocate 326.00 MiB (GPU 0; 2.00 GiB total capacity; 1.06 GiB already allocated; 65.44 MiB free; 1.09 GiB reserved in total by PyTorch)

In [15]:
print(np.mean(acc_list_2))

nan
C:\Users\Hanger\Anaconda3\envs\torch\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Hanger\Anaconda3\envs\torch\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [376]:
acc_list = []
for i in range(5):
    model = FBGAT(48, 8).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.05, weight_decay=5e-5)
    model.train()
    print("Number of Trial: " + str(i))
    reset_parameter(model)
    train_mask, test_mask = data_sample(num_nodes)
    for epoch in range(100):
        if epoch % 20 == 0:
            print(epoch)
        optimizer.zero_grad()
        out = model(data, adj, degree)
        loss = F.nll_loss(out[train_mask], data.y[train_mask]) 
        loss.backward()
        optimizer.step()
    model.eval()
    _, pred = model(data, adj, degree).max(dim=1)
    correct = int(pred[test_mask].eq(data.y[test_mask]).sum() .item())
    acc = correct / int(test_mask.sum())
    acc_list.append(acc)

Number of Trial: 0
0
20
40
60
80
Number of Trial: 1
0
20
40
60
80


In [378]:
print('FBGAT Accuracy: {:.4f}'.format(np.mean(acc_list)))

FBGAT Accuracy: 0.8579


In [362]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data)

aL_1 tensor([0.2276], device='cuda:0')
aH_1 tensor([0.0994], device='cuda:0')
aL_2 tensor([0.4996], device='cuda:0')
aH_2 tensor([0.3774], device='cuda:0')
fb1.low.weight tensor([[-2.0678e-01,  1.8474e-02,  5.6937e-03,  ...,  2.4587e-01,
          7.2109e-02, -1.6569e-01],
        [-1.3639e-03, -6.6650e-05,  4.0031e-05,  ...,  1.0259e-03,
          2.8664e-03, -1.0082e-03],
        [-9.0461e-02, -1.8909e-02,  4.0823e-02,  ...,  2.2229e-02,
         -2.6266e-01, -2.0773e-01],
        ...,
        [ 2.1201e-01,  2.5107e-02, -4.7889e-02,  ..., -1.5644e-01,
          5.5418e-02, -9.4412e-02],
        [-1.0996e-01,  2.2267e-03,  3.1053e-04,  ...,  1.3089e-01,
          3.8182e-02, -6.5425e-02],
        [-2.1739e-03,  7.7339e-05, -2.1302e-03,  ..., -9.4681e-04,
         -2.0912e-02, -6.9897e-04]], device='cuda:0')
fb2.low.weight tensor([[-0.3223,  0.5569, -0.4501,  ..., -0.1506, -0.1959, -0.3345],
        [ 0.1219,  0.2431,  0.0175,  ..., -0.2430,  0.3025, -0.3304],
        [ 0.2844,  0.2167